# Segmenting and Clustering Neighborhoods in Toronto <a class="tocSkip">

For the Toronto neighborhood data, a [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) exists that has all the information we need to explore and cluster the neighborhoods in Toronto. In this notebook we will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset. Once the data is in a structured format, we will explore and cluster the neighborhoods in the city of Toronto.

# Scraping the Wikipedia page

## 1. Assignment Requirements

Requirements:
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## 2. Parsing the Web Page

First we need to import libraries that we will use. After that the next step is to get the source code of the Wikipedia page with a call to `requests.get().text` and parse it with the `BeautifulSoup()` constructor creating a BeautifulSoup object.

In [1]:
# import libraries we will need

import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
# get the source code from the wikipedia page and pass it to the beautifulsoup constructor

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

soup = BeautifulSoup(source)

Now after examining the source code of the Wikipedia page, we know that the table of postal codes of Toronto is contained within the first `<table>` tag, so we can navigate to it using the table name as an attribute i.e. `soup.table`. After further examination we know, that table rows with the information we need are each contained within `<tr>` tags and each separate cell of that table row is further contained within `<td>` tag (`<th>` tag for the headers row). We can navigate to those tags and loop over them with `find_all()` method.

In [3]:
# extract the table of postal codes and iterate through the first 3 rows and check the results

table = soup.table

for row in table.find_all('tr', limit=3):
    print(row)
    print()

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>

<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>



We can loop further over separate cells calling `find_all()` method on each table row. In order to grab the text part of the tags we can access the `text` attribute of those tags. We can also store that text part of the cells into a list, so each row of the original table from the Wikipedia page is transformed into a list containing values of separate cells. This will be useful further when we transform that data into a pandas dataframe.      

In [4]:
# loop over the first 5 rows and their cells and extract the text part and check the results

for row in table.find_all('tr', limit=5):
    row_list = []
    
    for cell in row.find_all('td'):
        row_list.append(cell.text)
    
    print(row_list)

[]
['M1A', 'Not assigned', 'Not assigned\n']
['M2A', 'Not assigned', 'Not assigned\n']
['M3A', 'North York', 'Parkwoods\n']
['M4A', 'North York', 'Victoria Village\n']


This looks good. Looks like that code does what we expect it to do. We now can transform that data into a pandas dataframe as per requirements in the assignment. We will just need to delete that new line character `'\n'` from the last item of each list using `str.replace()` method. 

## 3. Creating Pandas Dataframe and Data Wrangling

### 3.1. Creating Dataframe

First let's create an empty pandas dataframe consisting of three required columns: **PostalCode**, **Borough**, and **Neighborhood**.

In [5]:
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_df = pd.DataFrame(columns=columns)

toronto_df

,PostalCode,Borough,Neighborhood


### 3.2. Populating Dataframe with Data from Wikipedia Page

Then we need to loop over rows and cells from the Wikipedia page using the code above and to assign the text from those cells to our dataframe cells using `pandas.DataFrame.loc[]` method.    

In [6]:
# iterate through rows and their indices with enumerate() function and populate our pandas dataframe 

for i, row in enumerate(table.find_all('tr')):
    row_list = []
    
    for cell in row.find_all('td'):
        row_list.append(cell.text.replace('\n', ''))
        
    # check that row_list is not empty (it is empty for the very first row from Wikipedia page with headers which we don't need)
    # then populate the next dataframe row with the data 
    if row_list: 
        toronto_df.loc[i, :] = row_list

toronto_df.head(10)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


Great, now we have copied all the information from the Wikipedia table into our `toronto_df` dataframe. We can now manipulate with it as per assignment requirements.

### 3.3. Extracting Rows with Assigned Boroughs

First we extract all the rows where **Borough is assigned** by using `pandas.DataFrame.loc[]` method.

In [7]:
toronto_df = toronto_df.loc[toronto_df['Borough'] != 'Not assigned']
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### 3.4. Assigning Names for Neighborhoods

Then we need to check which rows have **Neighborhood not assigned**. As you can see below there is only one such row.

We could set a value for Neighborhood column of that row manually, but the best practice would be the following. We can first define an `assign_neighborhood()` function and then apply it to the rows of our `toronto_df` dataframe.   

In [8]:
toronto_df.loc[toronto_df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
9,M7A,Queen's Park,Not assigned


In [9]:
def assign_neighborhood(row):
    
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough'] 
    
    return row

toronto_df = toronto_df.apply(assign_neighborhood, axis=1)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [10]:
# let's check that toronto_df does not contain any rows with neighbourhoods not assigned
toronto_df.loc[toronto_df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


### 3.5. Combining Neighborhoods with Same Postal Code

The last formatting we need to do is to combine Neighborhoods from the same postal code area into one row with the neighborhoods separated with a comma. For that we will use `pandas.DataFrame.groupby()` operation and then we use aggegation transformation with `agg()` method on that groupby object.

Within `agg()` method we will use `lambda` function which returns a set of distinct items (so that the same Borough will be returned just once while different Neighborhoods will be listed in a set). Then we apply `str.join()` method on those sets to return a string as an element of our dataframe.   

In [11]:
toronto_df = toronto_df.groupby(by='PostalCode', sort=False).agg(lambda x: ', '.join(set(x)))
toronto_df.head(10)

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge, Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens, Parkview Hill"


In [12]:
# let's reset the index column

toronto_df.reset_index(inplace=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### 3.6. Printing Number of Rows

In [13]:
print('The resulted dataframe has {} rows'.format(toronto_df.shape[0]))

The resulted dataframe has 103 rows
